In [126]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [127]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth', 'Utils', 'Other', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = '10 Industry Portfolios - Average Value Weighted Returns.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [128]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    rf = pd.read_csv(file_name, delimiter = ',', header = 0, usecols = ['RF'])
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = np.subtract(data,rf)
    
actives = list(returns_data.columns.values)
returns_data.head() # Eksempel på den procentvise ændring

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
date,,,,,,,,,,,
196307,-0.74,-0.49,-1.68,2.02,-0.96,-0.50,-1.30,0.29,0.53,-1.88,-0.39
196308,4.62,6.30,5.95,3.68,4.89,4.04,6.15,9.31,3.95,5.24,5.07
196309,-1.95,-0.51,-1.03,-3.91,-0.14,2.09,0.69,-4.33,-2.77,-3.43,-1.57
196310,2.37,9.43,2.29,-0.61,8.00,3.11,0.19,3.09,-0.96,1.09,2.53
196311,-1.40,-5.11,0.03,-1.42,-0.56,3.89,-1.50,-1.92,-1.29,-0.04,-0.85


In [129]:
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

In [130]:
# simulerer 1/N portfølje fra 1963-07 til 2004-11 som rollover med 60 måneders interval:
for month in range(0, len(returns_data) - 60):
    
    # Danner nyt data interval, returns og cov-matrix for hvert interval
    new_returns_data = returns_data[month:month + 60]
    returns_monthly = new_returns_data
    returns_mean = returns_monthly.mean()
    cov_matrix = returns_monthly.cov()
    
    # udregner profit, volatilitet og sharpe
    tl = np.dot(np.linalg.inv(cov_matrix), np.ones(11))
    nl = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), np.ones(11)))
    weights = tl / nl
    
    # udregner afkast for måned T + 1 med de udregnede optimale vægte
    returns = np.dot(weights, returns_data[month + 60:month + 61].mean())
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = returns / volatility

    # indsætter overstående udregninger i vektorerne
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

In [131]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

In [132]:
# definerer de enkelte vægte af de valgte aktiver, HINT: de er 0.33 alle sammen da det er 1/N
for i in range(len((actives))):
    symbol = actives[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [133]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in actives]
df = df[column_order]

df.head() # Porteføljer ud fra 60 måneders interval
          # ie. portfølje '0' har return, vol og sharpe ud fra måned 1 - 60
          # portfølje '1' har således overstående ud fra måned 2 - 61 ovs.

,Returns,Volatility,Sharpe Ratio,NoDur Weight,Durbl Weight,Manuf Weight,Enrgy Weight,HiTec Weight,Telcm Weight,Shops Weight,Hlth Weight,Utils Weight,Other Weight,Mkt Weight
0,0.513003,2.169277,0.236486,0.559951,-0.014613,0.665248,0.719823,0.393322,0.439056,0.239855,-0.067811,0.424580,-0.106193,-2.253217
1,0.866333,2.169095,0.399398,0.561129,-0.012219,0.674865,0.723241,0.396019,0.441355,0.242348,-0.067484,0.427143,-0.103102,-2.283295
2,0.914730,2.132046,0.429038,0.525887,0.039208,0.829502,0.794142,0.416071,0.487965,0.300829,0.028011,0.454393,-0.081051,-2.794956
3,0.389242,2.127410,0.182965,0.552380,0.039904,0.838225,0.818679,0.423537,0.479402,0.295415,0.048795,0.460881,-0.081186,-2.876031
4,6.301621,2.127361,2.962177,0.551688,0.037940,0.846815,0.823509,0.420057,0.479748,0.299253,0.050368,0.460040,-0.080104,-2.889314


In [134]:
df['Returns'].mean() / df['Returns'].std()

0.14952532144629926